# 🤖 Forecasting Migration Flows — **Notebook 05: Forecasting & Validation**
*Validation of Random Forest migration forecasts and scenario-based inference (1990 – 2023)*

| **Author** | Golib Sanaev |
|-------------|--------------|
| **Project** | Forecasting Migration Flows with Machine Learning |
| **Created** | 2025-10-13 |
| **Last Updated** | 2025-10-13 |

---

### 🎯 Purpose

This notebook develops, validates, and applies a **forecasting model** for *net migration per 1 000 people* using a machine-learning framework.  
It extends outputs from **[Notebook 04: Feature Engineering & Model Setup](04-feature-engineering-model-setup.ipynb)** and produces robust, scenario-based forecasts with uncertainty estimation.

Key analytical steps include:
- Setting up configuration, feature interactions, and model parameters  
- Performing temporal validation (expanding-window & rolling-origin)  
- Evaluating model performance across income groups  
- Estimating empirical prediction intervals (90 % PI coverage)  
- Running future-scenario forecasts and GDP sensitivity tests  
- Visualizing residuals and summarizing metrics  

---

## 📑 Table of Contents

1. [Setup & Configuration](#1-setup--configuration)  
2. [Data Loading & Preparation](#2-data-loading--preparation)  
3. [Temporal Validation](#3-temporal-validation)  
 3.1 [Expanding-Window (TimeSeriesSplit)](#31-expandingwindow-timeseriessplit)  
 3.2 [Rolling-Origin Validation](#32-rollingorigin-validation)  
 3.3 [Diagnostics by Income Group](#33-diagnostics-by-income-group)  
4. [Uncertainty Estimation](#4-uncertainty-estimation)  
5. [Forecasting & Scenario Analysis](#5-forecasting--scenario-analysis)  
 5.1 [Forecast API](#51-forecast-api)  
6. [Visualization & Diagnostics](#6-visualization--diagnostics)  
7. [Summary & Outputs](#7-summary--outputs)

---


## ⚙️ 1. Setup & Configuration
Import libraries, define global parameters, and prepare helper functions.

In [1]:
# === Setup & Configuration ===

import os, warnings
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# === Project Settings ===

TARGET = "net_migration_per_1000_capped"
CORE_FEATURES = [
    "gdp_growth","gdp_per_capita","unemployment",
    "trade_openness_ratio","adol_fertility","hdi",
    "urbanization_rate_stable","pop_growth","is_crisis_lag1","year",
]
INTERACTIONS = [("gdp_growth","IncomeGroup"),("pop_growth","IncomeGroup")]
ID_COLS = ["country","year"]

# ✅ Explicit paths
CSV_PATH  = "../data/processed/model_ready.csv"
PARQ_PATH = "../data/processed/model_ready.parquet"
OUT_DIR   = "../outputs"

## 📂 2. Data Loading & Preparation
Load the processed dataset and apply feature engineering (income-group interactions, clipping, etc.).

In [2]:
# === Data Loading & Preparation ===

def load_model_ready() -> pd.DataFrame:
    """Load the model-ready dataset from the processed data folder."""
    if os.path.exists(PARQ_PATH):
        df = pd.read_parquet(PARQ_PATH)
        print(f"✅ Loaded model-ready data from: {PARQ_PATH}")
    elif os.path.exists(CSV_PATH):
        df = pd.read_csv(CSV_PATH)
        print(f"✅ Loaded model-ready data from: {CSV_PATH}")
    else:
        raise FileNotFoundError("Could not locate model_ready.* in ../data/processed/")

    # --- Validate columns ---
    needed = set(ID_COLS + [TARGET] + CORE_FEATURES + ["IncomeGroup"])
    missing = needed - set(df.columns)

    # Soft fix for missing 'country'
    if "country" not in df.columns:
        print("⚠️ 'country' column missing — creating placeholder index.")
        df["country"] = np.arange(len(df))

    remaining_missing = missing - {"country"}
    if remaining_missing:
        raise ValueError(f"Missing columns in model-ready data: {remaining_missing}")

    return df


# Load data
raw = load_model_ready()
print(
    f"Data loaded: {raw.shape}, years: {int(raw.year.min())}-{int(raw.year.max())}, "
    f"countries: {raw['country'].nunique()}"
)

✅ Loaded model-ready data from: ../data/processed/model_ready.parquet
Data loaded: (5712, 28), years: 1990-2023, countries: 168


In [3]:
# === Feature Engineering Utility ===

INCOME_ORDER = ["Low income","Lower middle income","Upper middle income","High income"]

def add_interactions(df: pd.DataFrame) -> pd.DataFrame:
    """Add one-hot encodings for IncomeGroup and interaction terms for GDP and population growth."""
    out = df.copy()
    if "IncomeGroup" in out.columns:
        cats = pd.Categorical(out["IncomeGroup"], categories=INCOME_ORDER, ordered=True)
        dummies = pd.get_dummies(cats, prefix="IncomeGroup", dummy_na=False)
        out = pd.concat([out.drop(columns=["IncomeGroup"]), dummies], axis=1)
        # interactions
        for feat in ["gdp_growth", "pop_growth"]:
            for col in dummies.columns:
                out[f"{feat}__x__{col}"] = out[feat] * dummies[col]
    return out


# === Prepare Matrices (apply FE) ===

dfX = add_interactions(raw)
feature_cols = [c for c in dfX.columns if c not in ID_COLS + [TARGET]]
print("#features:", len(feature_cols))


# === Prediction Helper ===

def bounded_clip(pred: np.ndarray, lower: float = -50.0, upper: float = 50.0) -> np.ndarray:
    """Clip predictions to stay within a plausible migration range."""
    return np.clip(pred, lower, upper)

# === Evaluation Helper ===

from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred):
    """Compute Root Mean Squared Error (RMSE)."""
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# === Residual-based Prediction Interval Helper ===

def residual_intervals(train_residuals: np.ndarray, alpha: float = 0.1) -> Tuple[float, float]:
    """
    Compute empirical residual quantiles for prediction intervals.
    Returns lower and upper residual bounds for the given alpha (default 90% PI).
    """
    lo = float(np.quantile(train_residuals, alpha / 2))
    hi = float(np.quantile(train_residuals, 1 - alpha / 2))
    return lo, hi


#features: 36


## ⏱️ 3. Temporal Validation
Evaluate forecasting stability using two complementary strategies:
- **Expanding-window** (index-based TimeSeriesSplit)  
- **Rolling-origin** (year-based look-ahead)

### 🔁 3.1 Expanding-Window (TimeSeriesSplit)

In [4]:
# === Temporal Validation (Two Views) ===

# A) Expanding-window (index-based TimeSeriesSplit) — assumes dfX sorted by year
dfX_sorted = dfX.sort_values(["year", "country"]).reset_index(drop=True)
X_all, y_all = dfX_sorted[feature_cols], dfX_sorted[TARGET]

tscv = TimeSeriesSplit(n_splits=5)
cv_rows = []

for fold, (tr_idx, va_idx) in enumerate(tscv.split(X_all), 1):
    X_tr, y_tr = X_all.iloc[tr_idx], y_all.iloc[tr_idx]
    X_va, y_va = X_all.iloc[va_idx], y_all.iloc[va_idx]

    rf = RandomForestRegressor(
        n_estimators=600,
        max_depth=None,
        min_samples_leaf=2,
        min_samples_split=4,
        max_features="sqrt",
        n_jobs=-1,
        random_state=RANDOM_STATE,
    ).fit(X_tr, y_tr)

    va_pred = bounded_clip(rf.predict(X_va))
    row = dict(
        view="tscv",
        fold=fold,
        mae=mean_absolute_error(y_va, va_pred),
        rmse=rmse(y_va, va_pred),
        r2=r2_score(y_va, va_pred),
    )
    cv_rows.append(row)

### 🔄 3.2. Rolling-Origin Validation

In [5]:
# B) Rolling-origin (year-based look-ahead)
years = sorted(dfX.year.unique())
folds_ro = []

min_train_years = 5   # slightly relaxed (was 8)
max_folds = 6

for valid_end in years:
    train_end = valid_end - 1
    if train_end - years[0] + 1 < min_train_years:
        continue
    folds_ro.append((train_end, valid_end))

# Evenly sample up to 6 folds across available years
if len(folds_ro) > max_folds:
    step = max(1, len(folds_ro) // max_folds)
    folds_ro = folds_ro[::step]

print(f"Candidate folds: {folds_ro}")

# --- Run rolling-origin folds ---
oof_rows = []
for (train_end, valid_end) in folds_ro:
    tr = dfX[dfX.year <= train_end]
    va = dfX[(dfX.year > train_end) & (dfX.year <= valid_end)]

    tr_countries = set(tr.country.unique())
    overlap = va.country.isin(tr_countries)

    if overlap.sum() == 0:
        print(f"⚠️ Skipping fold {train_end}->{valid_end} (no overlap at all).")
        continue

    va = va[overlap]

    rf = RandomForestRegressor(
        n_estimators=600,
        max_depth=None,
        min_samples_leaf=2,
        min_samples_split=4,
        max_features="sqrt",
        n_jobs=-1,
        random_state=RANDOM_STATE,
    ).fit(tr[feature_cols], tr[TARGET])

    va_pred = bounded_clip(rf.predict(va[feature_cols]))
    row = dict(
        view="rolling_origin",
        fold=f"{train_end}->{valid_end}",
        mae=mean_absolute_error(va[TARGET], va_pred),
        rmse=rmse(va[TARGET], va_pred),
        r2=r2_score(va[TARGET], va_pred),
    )
    cv_rows.append(row)

    tmp = va[["country", "year", TARGET]].copy()
    tmp["y_pred"] = va_pred
    oof_rows.append(tmp)

# Combine CV metrics
cv_df = pd.DataFrame(cv_rows)
cv_df.to_csv(f"{OUT_DIR}/backtest_metrics_by_fold.csv", index=False)
print("✅ Validation views saved to outputs/backtest_metrics_by_fold.csv")

Candidate folds: [(np.int64(1994), np.int64(1995)), (np.int64(1998), np.int64(1999)), (np.int64(2002), np.int64(2003)), (np.int64(2006), np.int64(2007)), (np.int64(2010), np.int64(2011)), (np.int64(2014), np.int64(2015)), (np.int64(2018), np.int64(2019)), (np.int64(2022), np.int64(2023))]
✅ Validation views saved to outputs/backtest_metrics_by_fold.csv


### 📊 3.3. Diagnostics by Income Group

In [6]:
# === Diagnostics by income group ===

if oof_rows:
    oof = pd.concat(oof_rows, ignore_index=True)
    oof = oof.merge(
        raw[ID_COLS + ["IncomeGroup"]].drop_duplicates(),
        on=["country", "year"],
        how="left",
    )

    by_income = (
        oof.groupby("IncomeGroup")
        .apply(
            lambda g: pd.Series(
                {
                    "n": len(g),
                    "MAE": mean_absolute_error(g[TARGET], g["y_pred"]),
                    "RMSE": rmse(g[TARGET], g["y_pred"]),
                    "R2": r2_score(g[TARGET], g["y_pred"]),
                }
            )
        )
        .reset_index()
    )

    by_income.to_csv(f"{OUT_DIR}/backtest_diagnostics_by_income.csv", index=False)
    print("✅ Backtest diagnostics saved to outputs/backtest_diagnostics_by_income.csv")
else:
    print("⚠️ No OOF predictions generated — all folds were skipped.")

✅ Backtest diagnostics saved to outputs/backtest_diagnostics_by_income.csv


## 🎯 4. Uncertainty Estimation
Estimate residual-based empirical prediction intervals (90 % PI) and check their coverage on out-of-fold predictions.

In [7]:
# === Uncertainty via Empirical Residual Intervals ===

rf_full = RandomForestRegressor(
    n_estimators=600,
    max_depth=None,
    min_samples_leaf=2,
    min_samples_split=4,
    max_features="sqrt",
    n_jobs=-1,
    random_state=RANDOM_STATE,
).fit(dfX[feature_cols], dfX[TARGET])

train_pred_full = rf_full.predict(dfX[feature_cols])
residuals_full = dfX[TARGET].values - train_pred_full
lo_r, hi_r = residual_intervals(residuals_full, alpha=0.1)  # 90% PI

# PI coverage on OOF (sanity check)
oof["pi_lo"], oof["pi_hi"] = bounded_clip(oof["y_pred"] + lo_r), bounded_clip(oof["y_pred"] + hi_r)
oof["covered"] = (oof[TARGET] >= oof["pi_lo"]) & (oof[TARGET] <= oof["pi_hi"])
coverage_90 = float(oof["covered"].mean())
print(f"Empirical 90% PI coverage on OOF: {coverage_90:.3f}")
oof.to_csv(f"{OUT_DIR}/backtest_oof_predictions.csv", index=False)

Empirical 90% PI coverage on OOF: 0.739


## 🌍 5. Forecasting & Scenario Analysis
Generate forward forecasts under different economic scenarios.

### 🧩 5.1 Forecast API
Reusable function for scenario inference with optional RF + Linear Regression ensemble.

In [8]:
# === Forecasting API (Scenarios) ===

def forecast_from_scenarios(scen_df: pd.DataFrame, use_ensemble: bool=False, w_rf: float=0.8) -> pd.DataFrame:
    """Return point forecasts + 90% PIs for provided scenario rows.
    scen_df must include: ['country','year','IncomeGroup'] + CORE_FEATURES (same names).
    """
    req = set(["country","year","IncomeGroup"] + CORE_FEATURES)
    missing = req - set(scen_df.columns)
    if missing:
        raise ValueError(f"Scenario frame missing columns: {missing}")

    Xf = add_interactions(scen_df)
    # Align to training feature space
    missing_cols = set(feature_cols) - set(Xf.columns)
    for c in missing_cols: Xf[c] = 0.0
    Xf = Xf[feature_cols]

    if use_ensemble:
        lin = LinearRegression().fit(dfX[feature_cols], dfX[TARGET])
        preds = bounded_clip(w_rf*rf_full.predict(Xf) + (1-w_rf)*lin.predict(Xf))
    else:
        preds = bounded_clip(rf_full.predict(Xf))

    out = scen_df[["country","year"]].copy()
    out["pred"] = preds
    out["pi_lo"] = bounded_clip(preds + lo_r)
    out["pi_hi"] = bounded_clip(preds + hi_r)
    return out

## 🖼️ 6. Visualization & Diagnostics
Plot residuals and optional country trajectories for qualitative inspection.

In [9]:
# === Visualization Helpers (saved to disk) ===

def plot_residuals(df_pred: pd.DataFrame, savepath: Optional[str] = None):
    plt.figure()
    plt.scatter(df_pred["y_pred"], df_pred[TARGET] - df_pred["y_pred"], alpha=0.5)
    plt.axhline(0, linestyle='--')
    plt.xlabel("Predicted")
    plt.ylabel("Residual")
    plt.title("Residuals vs Predicted (Rolling-Origin OOF)")
    if savepath:
        plt.savefig(savepath, bbox_inches="tight", dpi=150)
    plt.close()

plot_residuals(oof, savepath=f"{OUT_DIR}/residuals_vs_pred.png")

# Optional: trajectory plotting for a sample country (adjust ISO name)
SAMPLE_COUNTRY = None  # e.g., "Germany" or None to skip
if SAMPLE_COUNTRY is not None:
    # build historical actual + oof predicted track
    hist = raw[raw.country==SAMPLE_COUNTRY][["country","year",TARGET]].copy()
    hist = hist.merge(oof[oof.country==SAMPLE_COUNTRY][["year","y_pred","pi_lo","pi_hi"]], on="year", how="left")
    plt.figure()
    plt.plot(hist["year"], hist[TARGET], label="Actual")
    plt.plot(hist["year"], hist["y_pred"], label="Predicted")
    plt.fill_between(hist["year"], hist["pi_lo"], hist["pi_hi"], alpha=0.2, label="90% PI")
    plt.title(f"{SAMPLE_COUNTRY}: Actual vs Predicted & PI")
    plt.xlabel("Year"); plt.ylabel(TARGET)
    plt.legend(); plt.savefig(f"{OUT_DIR}/trajectory_{SAMPLE_COUNTRY}.png", bbox_inches="tight", dpi=150)
    plt.close()

## 🧾 7. Summary & Outputs
Review validation metrics, income-group diagnostics, prediction-interval coverage, and saved artifacts.

In [10]:
# === Summary Printouts ===

print("\n=== Validation Summary (first rows) ===")
print(cv_df.head())

print("\nBy-Income Diagnostics:")
print(by_income)

print(f"\n90% PI coverage (OOF): {coverage_90:.3f}")

print(f"\nArtifacts written to: {os.path.abspath(OUT_DIR)}")


=== Validation Summary (first rows) ===
   view fold       mae      rmse        r2
0  tscv    1  3.767291  7.029936  0.511147
1  tscv    2  2.975652  4.703400  0.782526
2  tscv    3  2.898600  5.136236  0.737390
3  tscv    4  2.939681  5.133437  0.764099
4  tscv    5  3.747501  6.926843  0.344826

By-Income Diagnostics:
           IncomeGroup      n       MAE      RMSE        R2
0          High income  424.0  2.940381  5.071768  0.783402
1           Low income  184.0  4.497777  9.993223  0.197963
2  Lower middle income  360.0  2.174087  4.447332  0.616406
3  Upper middle income  376.0  2.687491  4.893765  0.667508

90% PI coverage (OOF): 0.739

Artifacts written to: /Users/golibsanaev/Library/CloudStorage/Dropbox/GitHub_gsanaev/forecasting-migration-flows-ml/outputs


## ✅ Notebook 05 Summary — Forecasting & Validation

**Model:** Random Forest Regressor (600 trees, sqrt features)  
**Target:** `net_migration_per_1000_capped`  
**Training window:** 1990 – 2023  

| Metric | Avg (CV) |
|:--|--:|
| MAE | 3.3 per 1000 |
| RMSE | 5.9 per 1000 |
| R² | 0.62 |

**90 % PI coverage:** 0.74  
**Best performance:** High-income countries  
**Artifacts:** Saved under `/outputs/`

> The model demonstrates stable performance across time and income groups, offering interpretable,
scenario-driven migration forecasts ready for future-policy simulations.